In [1]:
import urllib.request
import re
import pickle
from bs4 import BeautifulSoup

In [2]:
def gets_html(url, headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}):
    """ Lee un url y devuelve el código html """
    req = urllib.request.Request(url,headers=headers)
    return(urllib.request.urlopen(req).read().decode("utf-8"))

def find_links(soup,url_type):
    
    if url_type == "non_recipe":
        links = soup.find_all('a',class_="card__titleLink manual-link-behavior",href=True)
    elif url_type == "recipe":
        links = soup.find_all('a', {"data-click-id":[re.compile("simslot_\d")]},href=True)
    return links

def filter_recipes_links(links):
    
    RECIPE_REGEX = re.compile('href=\"https://www.allrecipes.com/recipe/\d+/.*?\"')
    
    # Convert to string 
    string_links = [str(i) for i in links]

    # Filter by regex
    filtered_links = list(set(filter(RECIPE_REGEX.search, string_links)))
    
    return filtered_links

def gets_links(links, n_links):
    """ Devuelve los primeros n links de un artículo """
    out = []
    while links and len(out) < n_links:
        li = links.pop(0)
        if 'http' in li:
            link_tmp = re.compile('href="(.+?)"').findall(li)[0]
        out.append(link_tmp)
    return(out)

def scraper(to_scrap, n_links, max_depth):

    html_data = {}
    link_regex = 'https://www.allrecipes.com/recipe/\d+'
    link_regex = re.compile(link_regex)
    i = 1
    while to_scrap:
        url, depth = to_scrap.pop()
        print('link:', url, i)
        if (url not in html_data) and (depth <= max_depth):
            if link_regex.match(url):
                URL_TYPE = "recipe"
            else:
                URL_TYPE = "non_recipe"
                
            html_data[url] = gets_html(url)
            soup = BeautifulSoup(html_data[url])
            
            links = find_links(soup,url_type=URL_TYPE)
            links = filter_recipes_links(links)
            
            page_urls = gets_links(links, n_links)
            page_urls = [(e, depth + 1) for e in page_urls]
            to_scrap.extend(page_urls)
            i +=1
            
    return(html_data)

In [3]:
#from requests_html import HTMLSession
#session = HTMLSession()

#r = session.get('https://www.allrecipes.com/recipes/79/desserts/')
#[(recipe,0) for recipe in r.html.links if "desserts" in recipe] + [('https://www.allrecipes.com/recipes/80/main-dish/',0)]

In [4]:
# Category page (should make something to list categories)
html = gets_html("https://www.allrecipes.com/recipes/193/holidays-and-events/new-year/")
soup = BeautifulSoup(html)
# Find links
links = soup.find_all('a',href=True)
# Convert to string 
links = [str(i) for i in links]

link_regex = 'href=\"https://www.allrecipes.com/recipes/\d+/.*new'
link_regex = re.compile(link_regex)

scrap_categories = []
# Filter by regex
links2 = list(filter(link_regex.search, links))
links2 = list(set(links2))

for li in links2:
    link_tmp = re.compile('href="(.+?)"').findall(li)[0]
    scrap_categories.append((link_tmp, 0))
    

In [5]:
scrap_categories 

[('https://www.allrecipes.com/recipes/17145/holidays-and-events/new-year/new-year-new-you/',
  0),
 ('https://www.allrecipes.com/recipes/193/holidays-and-events/new-year/', 0),
 ('https://www.allrecipes.com/recipes/1042/holidays-and-events/new-year/drinks-with-alcohol/',
  0),
 ('https://www.allrecipes.com/recipes/1274/holidays-and-events/new-year/side-dishes/',
  0),
 ('https://www.allrecipes.com/recipes/1272/holidays-and-events/new-year/desserts/',
  0),
 ('https://www.allrecipes.com/recipes/1273/holidays-and-events/new-year/main-dishes/',
  0),
 ('https://www.allrecipes.com/recipes/1041/holidays-and-events/new-year/appetizers/',
  0),
 ('https://www.allrecipes.com/recipes/1043/holidays-and-events/new-year/good-luck-foods/',
  0),
 ('https://www.allrecipes.com/recipes/193/holidays-and-events/new-year/?page=2',
  0),
 ('https://www.allrecipes.com/recipes/193/holidays-and-events/new-year/', 0),
 ('https://www.allrecipes.com/recipes/1275/holidays-and-events/new-year/drinks-without-alcoh

In [11]:
to_scrap = scrap_categories + [('https://www.allrecipes.com/',0)]
# Datos html
html_data = scraper(to_scrap, n_links=100, max_depth=20)

link: https://www.allrecipes.com/recipe/236476/mango-with-sticky-coconut-rice-kao-niaw/ 1
link: https://www.allrecipes.com/recipe/236413/pineapple-cleanser-smoothie/ 2
link: https://www.allrecipes.com/recipe/131520/crazy-caseys-long-island-spiced-tea/ 3
link: https://www.allrecipes.com/recipe/48150/a-different-carrot-raisin-salad/ 4
link: https://www.allrecipes.com/recipe/247327/low-calorie-root-beer-mocha-frappe/ 5
link: https://www.allrecipes.com/recipe/244216/anns-berry-green-smoothie/ 6
link: https://www.allrecipes.com/recipe/46350/shemakes-instant-chai-tea/ 7
link: https://www.allrecipes.com/recipe/32513/banana-fruit-smoothie/ 8
link: https://www.allrecipes.com/recipe/20686/basic-margarita/ 9
link: https://www.allrecipes.com/recipe/162613/green-smoothie/ 10
link: https://www.allrecipes.com/recipe/93275/blue-motorcycle/ 11
link: https://www.allrecipes.com/recipe/219076/chef-johns-turkey-chili/ 12
link: https://www.allrecipes.com/recipe/32611/coffee-whirl/ 13
link: https://www.allre

link: https://www.allrecipes.com/recipe/231354/creme-anglaise-sauce/ 108
link: https://www.allrecipes.com/recipe/219205/moms-carrot-and-raisin-salad/ 109
link: https://www.allrecipes.com/recipe/22738/banana-margaritas/ 110
link: https://www.allrecipes.com/recipe/228657/gelatinized-hot-chocolate/ 111
link: https://www.allrecipes.com/recipe/230644/persimmon-green-smoothie/ 112
link: https://www.allrecipes.com/recipe/235941/quick-hot-chocolate/ 113
link: https://www.allrecipes.com/recipe/236462/smores-in-a-cup/ 114
link: https://www.allrecipes.com/recipe/258692/simple-summer-java-float/ 115
link: https://www.allrecipes.com/recipe/237544/mint-julep-iced-tea/ 116
link: https://www.allrecipes.com/recipe/235313/marlenes-yogurt-berry-smoothie/ 117
link: https://www.allrecipes.com/recipe/228399/zesty-thai-cucumber-soup/ 118
link: https://www.allrecipes.com/recipe/265984/blackberry-margarita/ 119
link: https://www.allrecipes.com/recipe/258687/vanilla-blueberry-blended-coffee/ 120
link: https://w

link: https://www.allrecipes.com/recipe/256901/fuzzy-navel-shake/ 215
link: https://www.allrecipes.com/recipe/241037/manly-margarita/ 216
link: https://www.allrecipes.com/recipe/104945/coffee-smoother/ 217
link: https://www.allrecipes.com/recipe/229062/v9-juice/ 218
link: https://www.allrecipes.com/recipe/20533/electric-lemonade/ 219
link: https://www.allrecipes.com/recipe/246116/tamarind-agua-fresca/ 220
link: https://www.allrecipes.com/recipe/238004/rosy-ginger-peach-smoothie/ 221
link: https://www.allrecipes.com/recipe/236295/crawford-berry-smoothie/ 222
link: https://www.allrecipes.com/recipe/65639/red-lotus-cocktail/ 223
link: https://www.allrecipes.com/recipe/20629/kool-ade-punch/ 224
link: https://www.allrecipes.com/recipe/231460/tropical-carrot-raisin-salad/ 225
link: https://www.allrecipes.com/recipe/238129/mexican-peach/ 226
link: https://www.allrecipes.com/recipe/233937/orange-crush-fresh-squeezed-orange-and-vodka-cocktail/ 227
link: https://www.allrecipes.com/recipe/219292/

link: https://www.allrecipes.com/recipe/181891/sweet-lime-iced-tea/ 321
link: https://www.allrecipes.com/recipe/222347/mocha-punch-with-ice-cream/ 322
link: https://www.allrecipes.com/recipe/203220/green-tea-shake/ 323
link: https://www.allrecipes.com/recipe/244201/chef-johns-classic-strawberry-shortcake/ 324
link: https://www.allrecipes.com/recipe/232843/peach-sangria-with-raspberries/ 325
link: https://www.allrecipes.com/recipe/21458/blue-rita/ 326
link: https://www.allrecipes.com/recipe/145849/lemon-drop-iii/ 327
link: https://www.allrecipes.com/recipe/257298/peach-mango-sangria/ 328
link: https://www.allrecipes.com/recipe/215119/maxs-fresh-fruit-smoothie/ 329
link: https://www.allrecipes.com/recipe/273517/skinny-white-chocolate-coconut-iced-coffee/ 330
link: https://www.allrecipes.com/recipe/236465/peach-orange-iced-tea/ 331
link: https://www.allrecipes.com/recipe/239145/chef-johns-peach-pie/ 332
link: https://www.allrecipes.com/recipe/236221/green-drink-with-aloe-vera-juice/ 333
l

link: https://www.allrecipes.com/recipe/260575/chef-johns-almond-biscotti/ 427
link: https://www.allrecipes.com/recipe/222345/refreshing-summer-cucumber-lemonade/ 428
link: https://www.allrecipes.com/recipe/220726/creamy-avocado-cucumber-soup/ 429
link: https://www.allrecipes.com/recipe/279872/whipped-peanut-butter-iced-coffee/ 430
link: https://www.allrecipes.com/recipe/222556/black-russian-cocktail/ 431
link: https://www.allrecipes.com/recipe/244163/sweet-potato-and-banana-pancakes/ 432
link: https://www.allrecipes.com/recipe/244126/raspberry-cantaloupe-smoothie/ 433
link: https://www.allrecipes.com/recipe/222761/coconut-milk-rice-pudding/ 434
link: https://www.allrecipes.com/recipe/244313/turmeric-ginger-c-boost-life-juice/ 435
link: https://www.allrecipes.com/recipe/200623/saras-iced-coffee/ 436
link: https://www.allrecipes.com/recipe/232027/vanilla-banamango-smoothie/ 437
link: https://www.allrecipes.com/recipe/140558/pomegranate-martini/ 438
link: https://www.allrecipes.com/recip

link: https://www.allrecipes.com/recipe/260025/iced-coffee-perfection/ 533
link: https://www.allrecipes.com/recipe/20428/cool-off-smoothie/ 534
link: https://www.allrecipes.com/recipe/238525/shark-bite-cocktail/ 535
link: https://www.allrecipes.com/recipe/222550/blue-hawaiian-cocktail/ 536
link: https://www.allrecipes.com/recipe/241504/maries-healthy-breakfast-smoothie/ 537
link: https://www.allrecipes.com/recipe/246889/super-easy-peanut-butter-cookies/ 538
link: https://www.allrecipes.com/recipe/221143/the-ultimate-margarita/ 539
link: https://www.allrecipes.com/recipe/233323/ultimate-frozen-strawberry-jalapeno-margarita/ 540
link: https://www.allrecipes.com/recipe/146973/mixed-mocha/ 541
link: https://www.allrecipes.com/recipe/229280/russian-carrot-salad-korean-style/ 542
link: https://www.allrecipes.com/recipe/256896/chocolate-oatmeal-shake/ 543
link: https://www.allrecipes.com/recipe/237792/frozen-raspberry-margaritas/ 544
link: https://www.allrecipes.com/recipe/143032/arroz-con-le

link: https://www.allrecipes.com/recipe/240735/chocolate-cherry-christmas/ 639
link: https://www.allrecipes.com/recipe/247221/gooey-smores-dip/ 640
link: https://www.allrecipes.com/recipe/247053/thai-iced-tea-cha-yen/ 641
link: https://www.allrecipes.com/recipe/254739/dreamsicle-iced-tea-latte/ 642
link: https://www.allrecipes.com/recipe/20549/jamaica-coffee/ 643
link: https://www.allrecipes.com/recipe/247212/dragon-fruit-smoothie/ 644
link: https://www.allrecipes.com/recipe/237733/cherry-coconut-smoothie/ 645
link: https://www.allrecipes.com/recipe/239228/vegan-chocolate-hemp-high-fiber-smoothie/ 646
link: https://www.allrecipes.com/recipe/257402/sweet-banana-smoothie/ 647
link: https://www.allrecipes.com/recipe/256467/salted-caramel-pumpkin-latte/ 648
link: https://www.allrecipes.com/recipe/221150/chef-johns-chocolate-mint-brownies/ 649
link: https://www.allrecipes.com/recipe/244283/pineapple-carrot-smoothie/ 650
link: https://www.allrecipes.com/recipe/36441/almond-french-toast/ 651


In [12]:
len(html_data)

661

In [13]:
# Guardo los datos
with open("datasets/pickles/recipes_old_part5_2020_12.p", "wb") as p:
    pickle.dump(html_data, p)